In [1]:
import time
import torch
import sbibm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import io

from pydmd import DMDc
np.set_printoptions(suppress=True)

In [2]:
task_name = "two_moons"
device = "cpu"
task = sbibm.get_task(task_name)
prior = task.get_prior_dist()
simulator = task.get_simulator()

In [3]:
cached_fn = os.path.join("trained", f"{task_name}.nf")
with open(cached_fn, "rb") as f:
    encoder = pickle.load(f)
encoder.to(device)

Flow(
  (_transform): CompositeTransform(
    (_transforms): ModuleList(
      (0): PiecewiseRationalQuadraticCouplingTransform(
        (transform_net): ResidualNet(
          (initial_layer): Linear(in_features=3, out_features=50, bias=True)
          (blocks): ModuleList(
            (0-1): 2 x ResidualBlock(
              (context_layer): Linear(in_features=2, out_features=50, bias=True)
              (linear_layers): ModuleList(
                (0-1): 2 x Linear(in_features=50, out_features=50, bias=True)
              )
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (final_layer): Linear(in_features=50, out_features=29, bias=True)
        )
      )
      (1): LULinear()
      (2): PiecewiseRationalQuadraticCouplingTransform(
        (transform_net): ResidualNet(
          (initial_layer): Linear(in_features=3, out_features=50, bias=True)
          (blocks): ModuleList(
            (0-1): 2 x ResidualBlock(
              (context_layer):

In [41]:
mb_size = 10
test_theta = prior.sample((mb_size,))
test_x = simulator(test_theta)
As = np.hstack([test_theta, test_theta, test_theta, test_theta, test_theta])[:,:9].reshape((-1, 3, 3)) # n x n
Bs = np.hstack([test_theta, test_theta])[:,:3].reshape((-1, 3, 1)) # n x u

In [42]:
# m: trajectory length
def create_system_snapshots(As, Bs, m = 25):
    n = As.shape[-1]
    l = Bs.shape[-1]

    x0 = np.random.random((n, 1))
    u = np.random.rand(l, m - 1) - .5

    x0 = np.tile(x0, reps=(As.shape[0],1,1))
    u  = np.tile(u,  reps=(Bs.shape[0],1,1))

    snapshots = [x0]

    for i in range(m - 1):
        snapshots.append(As @ snapshots[i] + Bs @ u[:, :, i:i+1])
    snapshots = np.array(snapshots).T
    return {'snapshots': snapshots, 'u': u, 'B': Bs, 'A': As}

In [43]:
system = create_system_snapshots(As, Bs)

In [44]:
print(system['snapshots'].shape)
print(system['u'].shape)

(1, 3, 10, 25)
(10, 1, 24)


In [45]:
A_hats, B_hats = [], []
for i in range(mb_size):
    dmdc = DMDc(svd_rank=-1, opt=True)
    dmdc.fit(system['snapshots'][:,:,i,:], system['u'][i])
    A_hat, B_hat, _ = dmdc.reconstructed_data()
    A_hats.append(A_hat)
    B_hats.append(B_hat)

/home/yppatel/anaconda3/envs/chig/lib/python3.8/site-packages/pydmd/snapshots.py:72: UserWarning: Input data condition number 193412210.13076463. Consider preprocessing data, passing in augmented data
matrix, or regularization methods.
  warnings.warn(
/home/yppatel/anaconda3/envs/chig/lib/python3.8/site-packages/pydmd/snapshots.py:72: UserWarning: Input data condition number 10529503169.818735. Consider preprocessing data, passing in augmented data
matrix, or regularization methods.
  warnings.warn(
/home/yppatel/anaconda3/envs/chig/lib/python3.8/site-packages/pydmd/snapshots.py:72: UserWarning: Input data condition number 15796234407.15471. Consider preprocessing data, passing in augmented data
matrix, or regularization methods.
  warnings.warn(
/home/yppatel/anaconda3/envs/chig/lib/python3.8/site-packages/pydmd/snapshots.py:72: UserWarning: Input data condition number 2161225.6280447026. Consider preprocessing data, passing in augmented data
matrix, or regularization methods.
  warn

In [47]:
mb_idx = 3

print(f"A = {As[mb_idx]}")
print(f"A_hat = {np.real(A_hats[mb_idx])}")
print("------------------------")
print(f"B = {Bs[mb_idx]}")
print(f"B_hat = {np.real(B_hats[mb_idx])}")

A = [[ 0.75714064 -0.73988426  0.75714064]
 [-0.73988426  0.75714064 -0.73988426]
 [ 0.75714064 -0.73988426  0.75714064]]
A_hat = [[ 0.75714065 -0.73988423  0.75714065]
 [-0.73988427  0.75714062 -0.73988427]
 [ 0.75714065 -0.73988423  0.75714065]]
------------------------
B = [[ 0.75714064]
 [-0.73988426]
 [ 0.75714064]]
B_hat = [[ 0.75714064]
 [-0.73988426]
 [ 0.75714064]]


In [49]:
# done: 1. generate C
# done: 2. learn generative model for C^ (seems like it'll be identical since we can just consider C to be a reshaped vector)
# done: 3. simulate trajectories w/ C/C_hat (sort of separate, but can do concurrently)
# done: 4. recover C^ from trajetories 
# 5. learn optimal controller for C_hat